#Deep Learning con Python
`Autor: Erwing FC 
~erwingforerocastro@gmail.com`

El siguiente ejercicio es un clasificador de texto segun el sentimiento del mismo (positivo o negativo) en base a comentarios de IMDB.

In [ ]:
import os
from google.colab import files
import zipfile
#subimos el archivo guardado en external files / Natural Language processing
files.upload()
!unzip aclImdb.zip

Saving aclImdb.zip to aclImdb.zip


In [ ]:
imdb_dir='aclImdb'
train_dir=os.path.join(imdb_dir,'train') #archivo de entrenamiento

labels=[]
texts=[]


for label_type in ['neg','pos']:
  dir_name=os.path.join(train_dir,label_type) 
  for fname in os.listdir(dir_name):
    if fname[-4:]=='.txt': #si es un archivo de texto
      f=open(os.path.join(dir_name,fname)) #lo abrimos
      texts.append(f.read()) #lo guardamos en texts
      f.close()              #cerramos
      if label_type=='neg':  #guardamos la etiqueta correspondiente en labels
        labels.append(0)
      else:
        labels.append(1)
        
#el primer ejemplo:     
print(texts[0])
print(labels[0])

Mr Perlman gives a standout performance (as usual). Sadly, he has to struggle with an underwritten script and some nonsensical set pieces.<br /><br />Larsen is in "Die Hard" mode complete with singlet and bulging muscles, I'm sure he could do better but seems satisfied to grimace and snarl through his part.<br /><br />The lovely Erika is very decorative (even though fully clothed!) and shows some signs of "getting" acting at last.<br /><br />SFX are mainly poor CGI and steals from other movies.<br /><br />The shootouts are pitiful - worthy of the A-Team<br /><br />Not even worth seeing for Perlman - AVOID
0


In [ ]:
#recordemos que los embedding preentrenados son buenos en pocos datos de capacitación, en problemas mas especificos pueden no funcionar correctamente
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  #las reseñas seran solo de 100 
training_samples = 200  #ejemplos de entrenamiento 200
validation_samples = 10000 #ejemplos de validacion 10000
max_words = 10000 #las primeras 10000 palabras del diccionario

tokenizer=Tokenizer(num_words=max_words) # le asignamos al tokenizador un valor maximo de palabras de 10000
tokenizer.fit_on_texts(texts)            # le pasamos los textos para que extraiga los tokens  
sequences=tokenizer.texts_to_sequences(texts) #convertimos el texto en secuencias en base a los tokens anteriores ej: 

#texts='hola lola' tokenizer.word_index = {'a':0,'l':1,'h':2,'o':3} tokenizer.texts_to_sequences(texts)= [[2],[3],[1],[0],[],[1],[3],[1],[0]]

word_index=tokenizer.word_index
print('Encontrados %s unicos tokens.'%len(word_index))

data=pad_sequences(sequences,maxlen=maxlen) #rellenamos de 0'ros los datos que contengan menor dimensionalidad, y sin superar agrupaciones de 100 ej:
#agrupaciones de 3: [[1],[2],[3],[],[1]] pad_sequence= [[1,2,3],[0,0,1]]

labels=np.asarray(labels) #convertimos las etiquetas de array a numpy array
print('Tamaño del tensor de datos:', data.shape)
print('Tañamo de las etiquetas:', labels.shape)

#mezclamos los datos
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]

x_train = data[:training_samples] #200
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples] #de 200 hasta 10200
y_val = labels[training_samples: training_samples + validation_samples]

Using TensorFlow backend.


Encontrados 88582 unicos tokens.
Tamaño del tensor de datos: (25000, 100)
Tañamo de las etiquetas: (25000,)


Vaya a https://nlp.stanford.edu/projects/glove y descargue los embeddings precalculadas de Wikipedia en inglés de 2014. Es un archivo zip de 822 MB llamado glove.6B.zip, que contiene vectores de incrustación 100 dimensionales para 400,000 palabras (o sin palabras fichas) y descomprimirlo.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
embeddings_index={} 
f=open('glove.6B.100d.txt') 

for line in f:
  values=line.split() #limpiamos la linea de espacios
  word=values[0]      #traemos la palabra
  coefs=np.asarray(values[1:],dtype='float32') #el resto es el embedding de dicha palabra
  embeddings_index[word]=coefs                 #guardamos la palabra y le asignamos su embedding
f.close() #cerramos
print('Encontramos %s vectores de letras.'%len(embeddings_index))

Encontramos 400000 vectores de letras.


In [ ]:
#Preparación de la matriz de incrustaciones de palabras GloVe
embedding_dim=100
embedding_matrix=np.zeros((max_words,embedding_dim)) #creamos una matriz de 10000x100
for word,i in word_index.items(): #recorremos nuestros tokens (recordar que es un diccionario letra:numero)
  if i < max_words:               #solo hasta 100
    embedding_vector = embeddings_index.get(word) #traemos el vector perteneciente a la letra de nuestro embedding descargado
    if embedding_vector is not None:              #si el vector de dicha palabra existe:
        embedding_matrix[i] = embedding_vector    #lo guardamos en la matriz
#un ejemplo
print(embedding_matrix[1])

[-0.038194   -0.24487001  0.72812003 -0.39961001  0.083172    0.043953
 -0.39140999  0.3344     -0.57545     0.087459    0.28786999 -0.06731
  0.30906001 -0.26383999 -0.13231    -0.20757     0.33395001 -0.33848
 -0.31742999 -0.48335999  0.1464     -0.37303999  0.34577     0.052041
  0.44946    -0.46970999  0.02628    -0.54154998 -0.15518001 -0.14106999
 -0.039722    0.28277001  0.14393     0.23464    -0.31020999  0.086173
  0.20397     0.52623999  0.17163999 -0.082378   -0.71787    -0.41531
  0.20334999 -0.12763     0.41367     0.55186999  0.57907999 -0.33476999
 -0.36559001 -0.54856998 -0.062892    0.26583999  0.30204999  0.99774998
 -0.80480999 -3.0243001   0.01254    -0.36941999  2.21670008  0.72201002
 -0.24978     0.92136002  0.034514    0.46744999  1.10790002 -0.19358
 -0.074575    0.23353    -0.052062   -0.22044     0.057162   -0.15806
 -0.30798    -0.41624999  0.37972     0.15006    -0.53211999 -0.20550001
 -1.25259995  0.071624    0.70564997  0.49744001 -0.42063001  0.26148
 -

###Estructura del modelo

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense

net=Sequential()
net.add(Embedding(max_words,embedding_dim,input_length=maxlen))
net.add(Flatten())
net.add(Dense(32,activation='relu'))
net.add(Dense(1,activation='sigmoid'))
net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Carga de embeddings de palabras previamente formadas en la capa de incrustación
net.layers[0].set_weights([embedding_matrix])
net.layers[0].trainable = False
#evitamos que se entrene el embedding, por que ya descargamos uno anteriormente (recordar transfer learning)

###Entrenamiento y evaluación

In [ ]:
#agregamos la funcion de perdida y de optimizacion
net.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])
#entrenamos el modelo
history=net.fit(x_train,y_train,
                epochs=10,
                batch_size=32,
                validation_data=(x_val,y_val))
#guardamos los pesos
model.save_weights('pre_trained_glove_model.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(40,20))
plt.plot(epochs,acc,'bo',label='Pres. Entrenamiento')
plt.plot(epochs,val_acc,'b',label='Pres. Validación')
plt.title('Entrenamiento y validacion presición')
plt.legend()

In [ ]:
plt.figure(figsize=(40,20))
plt.plot(epochs, loss, 'bo', label='Perdida entrenamiento')
plt.plot(epochs, val_loss, 'b', label='Perdida validación')
plt.title('Entrenamiento y validacion Perdida')
plt.legend()
plt.show()

###Tokenizar los datos del conjunto de prueba

In [ ]:
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg','pos']: 
  dir_name=os.path.join(test_dir,label_type) 
  for fname in sorted(os.listdir(dir_name)): #organizamos los archivos 
       if fname[-4:]=='.txt':
         f=open(os.path.join(dir_name,fname))
         texts.append(f.read())
         f.close()
         if label_type=='neg':
           labels.append(0)
         else:
           labels.append(1)

sequences=tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [ ]:
#evaluar el modelo con el set de pruebas
net.load_weights('pre_trained_glove_model.h5')
net.evaluate(x_test,y_test)